# Agentic Workflows for Enterprise Resource Planning (ERP)

This notebook demonstrates how to implement intelligent AI agents that can autonomously manage various ERP processes, building on similar principles used in our calorie counter app's OpenAI integration.

## Overview

Unlike traditional ML approaches that simply classify or predict, agentic workflows involve AI systems that can:
- **Reason** about complex business scenarios
- **Plan** multi-step actions 
- **Execute** tasks autonomously
- **Adapt** based on outcomes
- **Collaborate** with other agents and humans

## ERP Domain Applications

We'll explore several key ERP use cases:
1. **Intelligent Invoice Processing Agent**
2. **Supply Chain Optimization Agent** 
3. **Financial Reporting Agent**
4. **Multi-Agent Procurement Workflow**

In [ ]:
# Required imports for our agentic ERP workflows
import json
import asyncio
import openai
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, asdict
from abc import ABC, abstractmethod
import uuid

# Set up OpenAI client (similar to our calorie counter app)
# Note: In production, use secure credential management like our iOS app
import os
openai.api_key = os.getenv('OPENAI_API_KEY', 'your-api-key-here')

print("Agentic ERP Workflow Environment Initialized")

## 1. Foundation: Agent Architecture

Let's start by building a foundational agent architecture that can be extended for various ERP tasks.

In [ ]:
@dataclass
class AgentMessage:
    """Represents a message in the agent communication system"""
    agent_id: str
    content: str
    message_type: str  # 'task', 'response', 'alert', 'update'
    timestamp: datetime
    metadata: Dict = None

@dataclass 
class ERPTask:
    """Represents a task in the ERP system"""
    task_id: str
    task_type: str
    priority: int  # 1 (highest) to 5 (lowest)
    data: Dict
    status: str = "pending"  # pending, processing, completed, failed
    assigned_agent: str = None
    created_at: datetime = None
    
    def __post_init__(self):
        if self.created_at is None:
            self.created_at = datetime.now()

class BaseERPAgent(ABC):
    """Base class for all ERP agents - similar to our MLModelManager pattern"""
    
    def __init__(self, agent_id: str, name: str):
        self.agent_id = agent_id
        self.name = name
        self.is_busy = False
        self.message_history = []
        self.capabilities = []
        
    @abstractmethod
    async def process_task(self, task: ERPTask) -> Dict:
        """Process an ERP task - must be implemented by subclasses"""
        pass
    
    async def send_message(self, recipient_id: str, content: str, message_type: str = "update"):
        """Send a message to another agent or system"""
        message = AgentMessage(
            agent_id=self.agent_id,
            content=content,
            message_type=message_type,
            timestamp=datetime.now()
        )
        self.message_history.append(message)
        return message
    
    def can_handle_task(self, task: ERPTask) -> bool:
        """Check if this agent can handle a specific task type"""
        return task.task_type in self.capabilities

print("Base agent architecture defined")

## 2. Intelligent Invoice Processing Agent

This agent demonstrates advanced document understanding and automated decision-making, extending the vision analysis concepts from our calorie counter app.

In [ ]:
class InvoiceProcessingAgent(BaseERPAgent):
    """Agent that processes invoices using vision and language understanding"""
    
    def __init__(self, agent_id: str = "invoice_agent"):
        super().__init__(agent_id, "Invoice Processing Agent")
        self.capabilities = ["invoice_processing", "document_analysis", "compliance_check"]
        self.approval_thresholds = {
            "auto_approve": 1000.0,  # Auto-approve under $1000
            "require_review": 10000.0  # Require human review over $10000
        }
    
    async def process_task(self, task: ERPTask) -> Dict:
        """Process an invoice with intelligent analysis"""
        self.is_busy = True
        
        try:
            if task.task_type == "invoice_processing":
                result = await self._analyze_invoice(task.data)
                await self._make_approval_decision(result)
                return result
            else:
                return {"error": f"Cannot handle task type: {task.task_type}"}
        finally:
            self.is_busy = False
    
    async def _analyze_invoice(self, invoice_data: Dict) -> Dict:
        """Analyze invoice using OpenAI's vision and reasoning capabilities"""
        
        # Simulate invoice analysis prompt (similar to our food analysis)
        analysis_prompt = f"""
        Analyze this invoice data and extract key information:
        {json.dumps(invoice_data, indent=2)}
        
        Return a JSON object with:
        {{
            "vendor_name": "<vendor_name>",
            "total_amount": <amount_as_number>,
            "currency": "<currency_code>",
            "invoice_date": "<YYYY-MM-DD>",
            "due_date": "<YYYY-MM-DD>",
            "line_items": [
                {{"description": "<item>", "quantity": <qty>, "unit_price": <price>, "total": <total>}}
            ],
            "compliance_issues": ["<list_any_issues_found>"],
            "confidence_score": <0.0_to_1.0>,
            "recommended_action": "<approve|review|reject>"
        }}
        
        Check for compliance issues like:
        - Missing required fields
        - Unusual amounts or pricing
        - Vendor not in approved list
        - Date inconsistencies
        """
        
        # Simulate OpenAI API call
        # In production, this would make actual API calls like our calorie counter
        await asyncio.sleep(1)  # Simulate processing time
        
        # Simulated response for demonstration
        analysis_result = {
            "vendor_name": invoice_data.get("vendor", "Acme Corp"),
            "total_amount": invoice_data.get("amount", 2500.00),
            "currency": "USD",
            "invoice_date": invoice_data.get("date", "2024-01-15"),
            "due_date": "2024-02-15",
            "line_items": [
                {"description": "Software License", "quantity": 1, "unit_price": 2000.00, "total": 2000.00},
                {"description": "Support Services", "quantity": 10, "unit_price": 50.00, "total": 500.00}
            ],
            "compliance_issues": [],
            "confidence_score": 0.92,
            "recommended_action": "review"  # Over auto-approve threshold
        }
        
        return analysis_result
    
    async def _make_approval_decision(self, analysis: Dict) -> str:
        """Make intelligent approval decisions based on business rules"""
        amount = analysis["total_amount"]
        confidence = analysis["confidence_score"]
        compliance_issues = analysis["compliance_issues"]
        
        # Decision logic
        if compliance_issues:
            decision = "reject"
            reason = f"Compliance issues found: {', '.join(compliance_issues)}"
        elif confidence < 0.8:
            decision = "review"
            reason = "Low confidence in analysis"
        elif amount <= self.approval_thresholds["auto_approve"]:
            decision = "approve"
            reason = "Amount within auto-approval threshold"
        elif amount >= self.approval_thresholds["require_review"]:
            decision = "review"
            reason = "Amount requires human review"
        else:
            decision = "approve"
            reason = "Standard approval criteria met"
        
        # Send notification to relevant stakeholders
        await self.send_message(
            "finance_team",
            f"Invoice decision: {decision.upper()} - {reason}",
            "alert"
        )
        
        analysis["approval_decision"] = decision
        analysis["decision_reason"] = reason
        
        return decision

# Create and test the invoice agent
invoice_agent = InvoiceProcessingAgent()
print(f"Created {invoice_agent.name} with capabilities: {invoice_agent.capabilities}")

In [ ]:
# Test the invoice processing agent
sample_invoice = ERPTask(
    task_id="INV-001",
    task_type="invoice_processing",
    priority=2,
    data={
        "vendor": "TechSolutions Inc",
        "amount": 2500.00,
        "date": "2024-01-15",
        "description": "Monthly software licensing and support"
    }
)

# Process the invoice
result = await invoice_agent.process_task(sample_invoice)
print("Invoice Processing Result:")
print(json.dumps(result, indent=2, default=str))

print("\nAgent Messages:")
for msg in invoice_agent.message_history:
    print(f"[{msg.timestamp}] {msg.message_type}: {msg.content}")

## 3. Supply Chain Optimization Agent

This agent demonstrates predictive analytics and autonomous decision-making for inventory management.

In [ ]:
class SupplyChainAgent(BaseERPAgent):
    """Agent that optimizes supply chain operations using predictive analytics"""
    
    def __init__(self, agent_id: str = "supply_chain_agent"):
        super().__init__(agent_id, "Supply Chain Optimization Agent")
        self.capabilities = ["inventory_optimization", "demand_forecasting", "supplier_management"]
        self.reorder_points = {}  # Dynamic reorder points by product
        self.supplier_performance = {}  # Track supplier reliability
    
    async def process_task(self, task: ERPTask) -> Dict:
        """Process supply chain optimization tasks"""
        self.is_busy = True
        
        try:
            if task.task_type == "inventory_optimization":
                return await self._optimize_inventory(task.data)
            elif task.task_type == "demand_forecasting":
                return await self._forecast_demand(task.data)
            elif task.task_type == "supplier_management":
                return await self._evaluate_suppliers(task.data)
            else:
                return {"error": f"Cannot handle task type: {task.task_type}"}
        finally:
            self.is_busy = False
    
    async def _optimize_inventory(self, inventory_data: Dict) -> Dict:
        """Optimize inventory levels using intelligent analysis"""
        
        # Simulate AI-powered analysis
        await asyncio.sleep(2)  # Simulate complex analysis time
        
        # Generate realistic optimization recommendations
        products = inventory_data.get("products", [])
        optimized_products = []
        
        for product in products:
            product_id = product["id"]
            current_stock = product["current_stock"]
            avg_demand = product.get("avg_monthly_demand", 100)
            lead_time = product.get("lead_time_days", 14)
            
            # Calculate intelligent reorder point (safety stock + lead time demand)
            safety_factor = 1.5  # 50% safety stock
            lead_time_demand = (avg_demand * lead_time) / 30
            reorder_point = int(lead_time_demand * safety_factor)
            
            # Economic Order Quantity (EOQ) approximation
            order_quantity = int(avg_demand * 0.75)  # Simplified EOQ
            
            # Determine urgency
            if current_stock <= reorder_point * 0.5:
                urgency = "critical"
            elif current_stock <= reorder_point:
                urgency = "high"
            elif current_stock <= reorder_point * 1.5:
                urgency = "medium"
            else:
                urgency = "low"
            
            optimized_products.append({
                "product_id": product_id,
                "current_stock": current_stock,
                "recommended_reorder_point": reorder_point,
                "recommended_order_quantity": order_quantity,
                "urgency": urgency,
                "reasoning": f"Based on {avg_demand}/month demand, {lead_time} day lead time"
            })
        
        result = {
            "products": optimized_products,
            "cost_savings_potential": sum(p.get("cost", 100) for p in products) * 0.15,
            "risk_assessment": "medium",
            "timestamp": datetime.now().isoformat()
        }
        
        # Alert for critical items
        critical_items = [p for p in optimized_products if p["urgency"] == "critical"]
        if critical_items:
            await self.send_message(
                "procurement_team",
                f"URGENT: {len(critical_items)} items need immediate reordering",
                "alert"
            )
        
        return result

# Create supply chain agent
supply_agent = SupplyChainAgent()
print(f"Created {supply_agent.name} with capabilities: {supply_agent.capabilities}")

## 4. Multi-Agent Procurement Workflow

This demonstrates how multiple agents can collaborate on complex ERP processes.

In [ ]:
class ERPWorkflowOrchestrator:
    """Orchestrates multi-agent workflows for complex ERP processes"""
    
    def __init__(self):
        self.agents = {}
        self.active_workflows = {}
        self.message_bus = []  # Simple message bus for agent communication
    
    def register_agent(self, agent: BaseERPAgent):
        """Register an agent with the orchestrator"""
        self.agents[agent.agent_id] = agent
    
    async def execute_procurement_workflow(self, procurement_request: Dict) -> Dict:
        """Execute a complex procurement workflow involving multiple agents"""
        workflow_id = str(uuid.uuid4())
        workflow_status = {
            "workflow_id": workflow_id,
            "status": "started",
            "steps": [],
            "start_time": datetime.now()
        }
        
        self.active_workflows[workflow_id] = workflow_status
        
        try:
            # Step 1: Supply chain agent analyzes inventory needs
            print("Step 1: Analyzing inventory requirements...")
            inventory_task = ERPTask(
                task_id=f"{workflow_id}-inventory",
                task_type="inventory_optimization",
                priority=1,
                data=procurement_request["inventory_data"]
            )
            
            inventory_analysis = await self.agents["supply_chain_agent"].process_task(inventory_task)
            workflow_status["steps"].append({
                "step": 1,
                "description": "Inventory analysis completed",
                "result": inventory_analysis,
                "timestamp": datetime.now()
            })
            
            # Step 2: Generate purchase requisitions for urgent items
            print("Step 2: Generating purchase requisitions...")
            urgent_items = [
                item for item in inventory_analysis["products"]
                if item["urgency"] in ["high", "critical"]
            ]
            
            workflow_status["steps"].append({
                "step": 2,
                "description": f"Generated {len(urgent_items)} purchase requisitions",
                "result": urgent_items,
                "timestamp": datetime.now()
            })
            
            # Final workflow summary
            workflow_status["status"] = "completed"
            workflow_status["end_time"] = datetime.now()
            workflow_status["summary"] = {
                "total_requisitions": len(urgent_items),
                "cost_savings_potential": inventory_analysis.get("cost_savings_potential", 0)
            }
            
            print(f"✅ Procurement workflow {workflow_id} completed successfully")
            
        except Exception as e:
            workflow_status["status"] = "failed"
            workflow_status["error"] = str(e)
            print(f"❌ Procurement workflow {workflow_id} failed: {e}")
        
        return workflow_status

# Create and configure the orchestrator
orchestrator = ERPWorkflowOrchestrator()
orchestrator.register_agent(invoice_agent)
orchestrator.register_agent(supply_agent)

print(f"Orchestrator configured with {len(orchestrator.agents)} agents")

## Key Benefits of Agentic ERP Workflows

### Compared to Traditional ML Approaches:

| **Traditional ML** | **Agentic Workflows** |
|-------------------|----------------------|
| Static predictions | Dynamic decision-making |
| Single-task focus | Multi-step process automation |
| Human interpretation required | Autonomous action capability |
| Reactive responses | Proactive planning |
| Isolated models | Collaborative intelligence |

### Real-World Impact:

1. **Reduced Processing Time**: Automated workflows reduce invoice processing from days to hours
2. **Improved Accuracy**: AI agents maintain consistent decision-making criteria
3. **Cost Optimization**: Intelligent inventory management reduces carrying costs by 15-25%
4. **Risk Mitigation**: Proactive identification of compliance issues and cash flow problems
5. **Scalability**: Agents can handle increasing transaction volumes without proportional staff increases

## Conclusion

This notebook demonstrated how agentic workflows transform traditional ERP processes from reactive, manual operations into intelligent, autonomous systems. By combining the vision analysis capabilities we use in our calorie counter app with sophisticated reasoning and planning, we can create AI agents that not only analyze data but take meaningful actions.

The key innovation is moving beyond simple classification or prediction to agents that can:
- Understand complex business contexts
- Make autonomous decisions within defined parameters  
- Coordinate with other agents for complex workflows
- Adapt their behavior based on outcomes
- Communicate effectively with humans and systems

This represents the next evolution in enterprise AI, where systems become true digital workers rather than just analytical tools.